In [1]:
%cd ../..
# %pip install .
import time
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

c:\Users\youse\Desktop\Research\Apperception\apperception


In [2]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'pedestrian'
) as pedestrians

select *
from pedestrians as t1
join Cameras as cam on t1.cameraId = Cameras.id
where
  t1.heading < 180 and t1.heading > -180 and
  (contained(t1.traj, road_segment("roads"), cam.timestamp) or
   contained(t1.traj, road_segment("Intersections"), cam.timestamp)) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -70 OR
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > 70) AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_translation, cam.timestamp) < 135
"""

'\nwith (\n  select *\n  from item_traj\n  where item_traj.object_type = \'pedestrian\'\n) as pedestrians\n\nselect *\nfrom pedestrians as t1\njoin Cameras as cam on t1.cameraId = Cameras.id\nwhere\n  t1.heading < 180 and t1.heading > -180 and\n  (contained(t1.traj, road_segment("roads"), cam.timestamp) or\n   contained(t1.traj, road_segment("Intersections"), cam.timestamp)) and\n  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -70 OR\n  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > 70) AND\n  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND\n  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND\n  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND\n  viewAngle(t1.traj, cam.egoHeading, cam.ego_translation, cam.timestamp) < 135\n'

In [10]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter(lambda obj: obj.object_type == 'human.pedestrian.adult' or
                                 obj.object_type == 'human.pedestrian.wheelchair' or
                                 obj.object_type == 'human.pedestrian.personal_mobility' or
                                 obj.object_type == 'human.pedestrian.construction_worker' or
                                 obj.object_type == 'human.pedestrian.police_officer' or
                                 obj.object_type == 'human.pedestrian.child' or
                                 obj.object_type == 'human.pedestrian.stroller')
def pred(obj1, cam):
    return (
        obj1.heading < 180 and obj1.heading > -180 and
        (F.contained(obj1.traj, F.road_segment("road"), cam.timestamp) or
            F.contained(obj1.traj, F.road_segment("intersection"))) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam.ego, cam.timestamp) < 135
    )

# With road direction
world = world.filter("lambda obj1, cam: " +
        "(F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) or " + 
            "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp)) and " +
        "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or " + 
            "F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and " + 
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and " +
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 135")

# Without road direction
# world = world.filter("lambda obj1, cam: " +
#         "(F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) or " + 
#             "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp)) and " +
#         "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or " + 
#             "F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and "
#         "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
#         "F.view_angle(obj1, cam.ego, cam.timestamp) < 135")
start = time.time()

key = world.get_traj_key()

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_traj_key
get_traj_key 
        SELECT itemId FROM (
        SELECT DISTINCT table_0.*
        FROM (
        SELECT DISTINCT table_0.*
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE ((table_0.objectType='human.pedestrian.adult') OR (table_0.objectType='human.pedestrian.wheelchair') OR (table_0.objectType='human.pedestrian.personal_mobility') OR (table_0.objectType='human.pedestrian.construction_worker') OR (table_0.objectType='human.pedestrian.police_officer') OR (table_0.objectType='human.pedestrian.child') OR (table_0.objectType='human.pedestrian.stroller'))
        ) as table_0
        
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((contained(table_0.trajCentroids, roadSegment('road'), Cameras.timestamp) OR contained(table_0.trajCentroids, roadSegment('intersection'), Cameras.timestamp)) AND ((facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.time

In [11]:
import numpy as np

keys = world.get_traj_key()
trajs = world.get_traj()
scene_names = world.get_traj_attr("cameraId")
print(keys)
images = []
for i in range(len(keys)):
    key = keys[i][0]
    traj = trajs[i][0]
    scene_name = scene_names[i][0]
    current_images = world.get_trajectory_images(scene_name, traj)
    for image in current_images:
        images.append(image)

execute: filter
execute: filter
execute: get_traj_key
get_traj_key 
        SELECT itemId FROM (
        SELECT DISTINCT table_0.*
        FROM (
        SELECT DISTINCT table_0.*
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE ((table_0.objectType='human.pedestrian.adult') OR (table_0.objectType='human.pedestrian.wheelchair') OR (table_0.objectType='human.pedestrian.personal_mobility') OR (table_0.objectType='human.pedestrian.construction_worker') OR (table_0.objectType='human.pedestrian.police_officer') OR (table_0.objectType='human.pedestrian.child') OR (table_0.objectType='human.pedestrian.stroller'))
        ) as table_0
        
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((contained(table_0.trajCentroids, roadSegment('road'), Cameras.timestamp) OR contained(table_0.trajCentroids, roadSegment('intersection'), Cameras.timestamp)) AND ((facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.time

In [12]:
images_flat = np.array(images, dtype='object').flatten()
no_prefix_imgs = []
for image in images_flat:
    no_prefix_imgs.append(image.split("/")[2])

In [13]:
expected_imgs = ['n008-2018-05-21-11-06-59-0400__CAM_FRONT__1526915471412465.jpg', 'n008-2018-07-27-12-07-38-0400__CAM_FRONT__1532707917112404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153662404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154662404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385155162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385158662404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159662404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805162404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805662415.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806162404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806612404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807162404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807662404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656808162404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404362404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404762404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405262404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405762404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659406262404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535728830362404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326912404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327912404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328912404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329912795.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729330362404.jpg', 'n008-2018-08-31-11-37-23-0400__CAM_FRONT__1535730293412404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236162404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236662404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237112404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237612404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291162404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291662404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292162404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292662404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293162404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293662404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299143862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299149412404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299229362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299239112404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299240112404.jpg']

In [14]:
missing = 0
for expected_img in expected_imgs:
    if expected_img not in no_prefix_imgs:
        missing += 1
print("Percentage of expected results missing from query: ", 100 * missing / len(expected_imgs), "%")

extra = 0
for img in no_prefix_imgs:
    if img not in expected_imgs:
        extra += 1
print("Percentage of images in query but not in expected results: ", 100 * extra / len(no_prefix_imgs), "%")

Percentage of expected results missing from query:  86.20689655172414 %
Percentage of images in query but not in expected results:  58.97435897435897 %
